In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:
# Files to Load 
enroll1516_data_to_load = "SCHOOL_ENROLL.xls"
PIpercent1516_data_to_load = "1516_BUILDING_ACHIEVEMENT.xls"
VA1516_data_to_load = "1516_VA_org_DETAILS.xls"

In [3]:
enroll1516_data = pd.read_excel(enroll1516_data_to_load,sheet_name=[0, 1, 2])
PIpercent1516_data = pd.read_excel(PIpercent1516_data_to_load)
VA1516_data = pd.read_excel(VA1516_data_to_load)

In [4]:
#Function that filters data by District
def data_filter(df):
    return df[df["District IRN"] == 43786]

In [5]:
enroll1516_df0= data_filter(enroll1516_data[0])
enroll1516_df1= data_filter(enroll1516_data[1])

In [6]:
PIpercent1516_df=data_filter(PIpercent1516_data)

In [7]:
VA1516_df=data_filter(VA1516_data)

In [8]:
 #replace NC values with zero
enroll1516_df0.replace('NC', '0', inplace=True)
enroll1516_df1.replace('NC', '0', inplace=True)
VA1516_df.replace('NC', '0', inplace=True)
PIpercent1516_df.replace('NC', '0', inplace=True)
    
 #replace <10 value with zero
enroll1516_df0.replace('^.*<.*$', '0', regex=True, inplace=True)
enroll1516_df1.replace('^.*<.*$', '0', regex=True, inplace=True)
VA1516_df.replace('^.*<.*$', '0', regex=True, inplace=True)
PIpercent1516_df.replace('^.*<.*$', '0', regex=True, inplace=True)

In [9]:
# Function to concatenate all DataFrames from each Excel sheet into one single pandas DataFrame
def concat_function(df1, df2):
    df_concat1 = pd.concat([df1, df2])
    return pd.concat([df_concat1])

In [10]:
#Run concat function for 2015-2016,2016-2017 data
cmsd_enroll1516 = concat_function(enroll1516_df0, enroll1516_df1)

In [11]:
#Convert values into integers for Enrollment columns only
cmsd_enroll1516= cmsd_enroll1516.astype({'Pre-School Enrollment': 'int', 'Kindergarten Enrollment':'int',
       'First Grade Enrollment': 'int', 'Second Grade Enrollment': 'int',
       'Third Grade Enrollment': 'int', 'Fourth Grade Enrollment': 'int',
       'Fifth Grade Enrollment':'int', 'Sixth Grade Enrollment': 'int',
       'Seventh Grade Enrollment':'int', 'Eighth Grade Enrollment':'int',
       'Ninth Grade Enrollment ': 'int', 'Tenth Grade Enrollment': 'int',
       'Eleventh Grade Enrollment': 'int', 'Twelfth Grade Enrollment': 'int',
       'Thirteenth Grade Enrollment': 'int', 'Enrollment Past Twelfth Grade (Students with Disabilities)': 'int'})

In [12]:
#Convert values into integers for Performance Index Percent column
PI1516_df=  PIpercent1516_df.astype({'Performance Index Percent 2015-16':'float'})

In [13]:
#create a new column for enrollment total and use iloc to add from last column Preschool enrollment
cmsd_enroll1516['Enrollment 2015-2016'] = cmsd_enroll1516.iloc[:, -17:-1].sum(1)

In [14]:
#Keep necessary columns, drop all others
cmsd_enroll1516 = cmsd_enroll1516[['Building Name','Building IRN','Enrollment 2015-2016']]
VA1516_df= VA1516_df[['Building Name','Building IRN','Overall Value Added Grade']]
PI1516_df= PI1516_df[['Building Name','Building IRN','Performance Index Percent 2015-16']]

In [15]:
# Function to merge three dataframes into a single DataFrame
from functools import reduce
dfl=[cmsd_enroll1516, PI1516_df, VA1516_df]
cmsd_combined = reduce(lambda  left,right: pd.merge(left,right,on=['Building IRN'],how='outer'), dfl)

In [16]:
cmsd_combined

,Building Name_x,Building IRN,Enrollment 2015-2016,Building Name_y,Performance Index Percent 2015-16,Building Name,Overall Value Added Grade
0,Adlai Stevenson School,224,378,Adlai Stevenson School,36.6,Adlai Stevenson School,F
1,Almira,489,424,Almira,38.7,Almira,F
2,Andrew J Rickoff,729,406,Andrew J Rickoff,36.9,Andrew J Rickoff,F
3,Anton Grdina,828,325,Anton Grdina,32.4,Anton Grdina,F
4,Artemus Ward,1040,441,Artemus Ward,47.3,Artemus Ward,F
5,Benjamin Franklin,2378,533,Benjamin Franklin,54.1,Benjamin Franklin,F
6,Bolton,3137,300,Bolton,30.5,Bolton,F
7,Buhrer,4234,346,Buhrer,59.8,Buhrer,F
8,Case,5066,315,Case,32.3,Case,F
9,Alfred Benesch,5637,325,Alfred Benesch,32.7,Alfred Benesch,F


In [17]:
#Create function that drops _y suffixes
def drop_y(df):
    # list comprehension of the columns that end with _y
    to_drop = [x for x in df if x.endswith('_y')]
    df.drop(to_drop, axis=1, inplace=True)

In [18]:
#Create function that drops _x suffixes
def drop_x(df):
    # list comprehension of the columns that end with _x
    to_drop = [x for x in df if x.endswith('_x')]
    df.drop(to_drop, axis=1, inplace=True)

In [19]:
#Run drop_y function for 1516 and 1617 data
drop_x(cmsd_combined)
drop_y(cmsd_combined)

In [20]:
cmsd_combined

,Building IRN,Enrollment 2015-2016,Performance Index Percent 2015-16,Building Name,Overall Value Added Grade
0,224,378,36.6,Adlai Stevenson School,F
1,489,424,38.7,Almira,F
2,729,406,36.9,Andrew J Rickoff,F
3,828,325,32.4,Anton Grdina,F
4,1040,441,47.3,Artemus Ward,F
5,2378,533,54.1,Benjamin Franklin,F
6,3137,300,30.5,Bolton,F
7,4234,346,59.8,Buhrer,F
8,5066,315,32.3,Case,F
9,5637,325,32.7,Alfred Benesch,F


In [21]:
#Add new column School Year
cmsd_combined_df= cmsd_combined.assign(School_Year = '2015-2016')[['School_Year']+cmsd_combined.columns.tolist()]

In [22]:
cmsd_combined_df

,School_Year,Building IRN,Enrollment 2015-2016,Performance Index Percent 2015-16,Building Name,Overall Value Added Grade
0,2015-2016,224,378,36.6,Adlai Stevenson School,F
1,2015-2016,489,424,38.7,Almira,F
2,2015-2016,729,406,36.9,Andrew J Rickoff,F
3,2015-2016,828,325,32.4,Anton Grdina,F
4,2015-2016,1040,441,47.3,Artemus Ward,F
5,2015-2016,2378,533,54.1,Benjamin Franklin,F
6,2015-2016,3137,300,30.5,Bolton,F
7,2015-2016,4234,346,59.8,Buhrer,F
8,2015-2016,5066,315,32.3,Case,F
9,2015-2016,5637,325,32.7,Alfred Benesch,F


In [23]:
grouped_date_df = cmsd_combined_df.groupby(['School_Year','Building Name', 'Overall Value Added Grade'])

df5=grouped_date_df.mean()
df5.head(20)

Building IRN  \
School_Year Building Name                Overall Value Added Grade                 
2015-2016   Adlai Stevenson School       F                                   224   
            Alfred Benesch               F                                  5637   
            Almira                       F                                   489   
            Andrew J Rickoff             F                                   729   
            Anton Grdina                 F                                   828   
            Artemus Ward                 F                                  1040   
            Bard Early College Cleveland F                                 14920   
            Benjamin Franklin            F                                  2378   
            Bolton                       F                                  3137   
            Buhrer                       F                                  4234   
            Campus International School  F                                 12350   
            Case                         F                                  5066   
            Charles A Mooney School      F                                  5892   
            Charles Dickens School       F                                  5900   
            Charles W Eliot School       A                                  5942   
            Citizens Academy             F                                133520   
            Citizens Academy East        F                                 12852   
            Citizens Academy Southeast   NR                                15261   
            Citizens Leadership Academy  A                                 12029   
            Clara E Westropp School      F                                  6353   

                                                                    Enrollment 2015-2016  \
School_Year Building Name                Overall Value Added Grade                         
2015-2016   Adlai Stevenson School       F                                           378   
            Alfred Benesch               F                                           325   
            Almira                       F                                           424   
            Andrew J Rickoff             F                                           406   
            Anton Grdina                 F                                           325   
            Artemus Ward                 F                                           441   
            Bard Early College Cleveland F                                           249   
            Benjamin Franklin            F                                           533   
            Bolton                       F                                           300   
            Buhrer                       F                                           346   
            Campus International School  F                                           464   
            Case                         F                                           315   
            Charles A Mooney School      F                                           461   
            Charles Dickens School       F                                           337   
            Charles W Eliot School       A                                           318   
            Citizens Academy             F                                           376   
            Citizens Academy East        F                                           306   
            Citizens Academy Southeast   NR                                           70   
            Citizens Leadership Academy  A                                           258   
            Clara E Westropp School      F                                           352   

                                                                    Performance Index Percent 2015-16  
School_Year Building Name                Overall Value Added Grade                        

In [24]:
df5.to_excel("cmsd1516.xls")